In [1]:
%cd ..
from tqdm import tqdm
from utils.data import preprocess_event_database
import numpy as np
import torch
import pickle
import os

from torch_geometric.data import HeteroData
from relbench.datasets import get_dataset

device = torch.device('cuda:6' if torch.cuda.is_available() else 'cpu')

/home/lingze/embedding_fusion


/home/lingze/anaconda3/envs/deepdb/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset = get_dataset('rel-event')
db = dataset.get_db()
preprocess_event_database(db)

Loading Database object from /home/lingze/.cache/relbench/rel-event/db...
Done in 3.01 seconds.


/home/lingze/embedding_fusion/utils/data.py:231: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  event_df["event_id"].replace(event_id2index, inplace=True)
/home/lingze/embedding_fusion/utils/data.py:231: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  event_df["event_id"].replace(event_id2index, inplace=True)
/home/lingze/embedding_fusion/utils/d

In [3]:
cache_path = "./data/rel-event-tensor-frame/"
# [NOTE]: the dataset has been materialized

# get infer_type in cache
type_path = os.path.join(cache_path,"col_type_dict.pkl")
col_type_dict = pickle.load(open(type_path, "rb"))
len(col_type_dict)

# add "compress_text" in each table in case 
for table_name, table in db.table_dict.items():
    table.df["text_compress"] = np.nan

In [4]:
from utils.resource import get_text_embedder_cfg
text_embedder_cfg = get_text_embedder_cfg(
    model_name = "sentence-transformers/average_word_embeddings_glove.6B.300d", 
    device = "cpu")

/home/lingze/anaconda3/envs/deepdb/lib/python3.9/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [5]:
from utils.builder import build_pyg_hetero_graph
data, col_stats_dict = build_pyg_hetero_graph(
    db,
    col_type_dict,
    text_embedder_cfg,
    cache_path,
    True,
)

-----> Materialize event_attendees Tensor Frame
-----> Build edge between users and users
-----> Materialize events Tensor Frame
-----> Materialize event_interest Tensor Frame
-----> Materialize users Tensor Frame


In [6]:
# get the relbench tasks
from relbench.tasks import get_task
from relbench.modeling.graph import get_node_train_table_input
from torch_geometric.loader import NeighborLoader
from relbench.base import BaseTask
from model.base import CompositeModel, FeatureEncodingPart, NodeRepresentationPart
from relbench.modeling.nn import HeteroTemporalEncoder
# start to fine-train on the task a
from torch.nn import BCEWithLogitsLoss
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import math
import copy

In [7]:
task_a = get_task("rel-event", "user-attendance", download = True)
entity_table = task_a.entity_table

In [8]:
def generate_loader_dict(task: BaseTask, data:HeteroData) -> dict:
    loader_dict = {}
    for split, table in [
        ("train", task.get_table("train")),
        ("val",task.get_table("val")),
        ("test", task.get_table("test")),
    ]:
        table_input = get_node_train_table_input(
            table=table,
            task=task,
        )
        loader_dict[split] = NeighborLoader(
            data,
            num_neighbors=[
                128, 64
            ],  # we sample subgraphs of depth 2, 128 neighbors per node.
            time_attr="time",
            input_nodes=table_input.nodes,
            input_time=table_input.time,
            transform=table_input.transform,
            batch_size=512,
            temporal_strategy="uniform",
            shuffle=split == "train",
            num_workers=0,
            persistent_workers=False,
        )
    return loader_dict

In [9]:
@torch.no_grad()
def test(loader: NeighborLoader, model: torch.nn.Module, task: BaseTask, early_stop: int = 0)-> np.ndarray:
    # model.eval()
    pred_list = []
    early_stop = early_stop if early_stop > 0 else len(loader)
    for idx,batch in tqdm(enumerate(loader), leave=False, total=len(loader)):
        if idx > early_stop:
            break
        batch = batch.to(device)
        pred = model(
            batch,
            task.entity_table,
        )
        pred = pred.view(-1) if pred.size(1) == 1 else pred
        pred_list.append(pred.detach().cpu())
    return torch.cat(pred_list, dim=0)

In [10]:
# construct bottom model
channels = 128
temporal_encoder = HeteroTemporalEncoder(
    node_types=[
                node_type for node_type in data.node_types if "time" in data[node_type]
            ],
    channels=channels,
)

feat_encoder = FeatureEncodingPart(
    data=data,
    node_to_col_stats=col_stats_dict,
    channels=channels
)

node_encoder = NodeRepresentationPart(
    data=data,
    channels=channels,
    num_layers=1,
    normalization="layer_norm",
    dropout_prob=0.2
)


net = CompositeModel(
    data=data,
    channels=channels,
    out_channels=1,
    dropout=0.2,
    aggr="sum",
    norm="batch_norm",
    num_layer=2,
    feature_encoder=feat_encoder,
    node_encoder=node_encoder,
    temporal_encoder=temporal_encoder
)

In [11]:
# for regression task, we need to deactivate the normalization and dropout layer
task_a.task_type
# freeze_instances = (torch.nn.BatchNorm1d, torch.nn.LayerNorm, torch.nn.Dropout, torch.nn.BatchNorm2d)
deactive_nn_instances = (torch.nn.Dropout, torch.nn.Dropout2d, torch.nn.Dropout3d)
net.train()
for module in net.modules():
    if isinstance(module, deactive_nn_instances):
        module.eval()
        for param in module.parameters():
            param.requires_grad = False


In [12]:
# training for fine-tune
from torch.nn import L1Loss
from sklearn.metrics import mean_absolute_error, r2_score, root_mean_squared_error

task_loader_dict = generate_loader_dict(task_a,data)
lr = 0.005
epoches = 80
loss_fn = L1Loss()
tune_metric = "mae"
higher_is_better = False
early_stop = 10
max_round_epoch = 20
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, net.parameters()), lr = lr)

In [13]:
state_dict = None
best_val_metric = -math.inf if higher_is_better else math.inf
net.to(device)
best_epoch = 0
patience = 0
test_early_stop = 50
# train
for epoch in range(1, epoches + 1):
    cnt = 0
    loss_accum = count_accum = 0
    # net.train()
    for batch in tqdm(task_loader_dict["train"], leave = False):
        cnt += 1
        if cnt > max_round_epoch:
            break
        batch = batch.to(device)
        optimizer.zero_grad()
        pred = net(
            batch,
            task_a.entity_table,
        )
        pred = pred.view(-1) if pred.size(1) == 1 else pred
        loss = loss_fn(pred, batch[task_a.entity_table].y.float())
        
        loss.backward()
        optimizer.step()
        
        loss_accum += loss.detach().item() * pred.size(0)
        count_accum += pred.size(0)

    train_loss = loss_accum / count_accum

    val_logits = test(task_loader_dict["val"], net, task_a, test_early_stop)
    val_logits = val_logits.numpy()
    val_n = len(val_logits)
    val_pred_hat = task_a.get_table("val").df[task_a.target_col].to_numpy()[:val_n]
    val_metrics = {
        "mae": mean_absolute_error(val_pred_hat, val_logits),
        # "r2": r2_score(val_pred_hat, val_logits),
        # "rmse": root_mean_squared_error(val_pred_hat, val_logits),
    }
    print(f"Epoch: {epoch:02d}, Train loss: {train_loss}, Val metrics: {val_metrics}")
    


    
    if (higher_is_better and val_metrics[tune_metric] > best_val_metric) or (
        not higher_is_better and val_metrics[tune_metric] < best_val_metric
    ):
        patience = 0
        best_epoch = epoch
        best_val_metric = val_metrics[tune_metric]
        state_dict = copy.deepcopy(net.state_dict())
        # calculate test metrics
        logits = test(task_loader_dict["test"], net, task_a, test_early_stop)
        logits = logits.numpy()
        test_n = len(logits)
        pred_hat = task_a.get_table("test", mask_input_cols=False).df[task_a.target_col].to_numpy()[:test_n]
        test_metrics = {
                "mae": mean_absolute_error(pred_hat, logits),
                # "r2": r2_score(pred_hat, logits),
                # "rmse": root_mean_squared_error(pred_hat, logits),
        }
        print(f"Update the best scores\t Test metrics: {test_metrics}")
    else:
        patience += 1
    
    if patience >= early_stop:
        print(f"Early stop at epoch {epoch}")
        break   
    
best_epoch

Epoch: 01, Train loss: 0.513465379178524, Val metrics: {'mae': 0.2890320514729899}


Update the best scores	 Test metrics: {'mae': 0.2916935517702849}


Epoch: 02, Train loss: 0.385674361884594, Val metrics: {'mae': 0.2839550400088956}


Update the best scores	 Test metrics: {'mae': 0.28764935529413155}


Epoch: 03, Train loss: 0.39104495048522947, Val metrics: {'mae': 0.2711382860181806}


...

Epoch: 15, Train loss: 0.3305986925959587, Val metrics: {'mae': 0.2516373836681047}


Epoch: 16, Train loss: 0.33554082959890363, Val metrics: {'mae': 0.2476508033004304}


Epoch: 17, Train loss: 0.33407666385173795, Val metrics: {'mae': 0.2594093536329814}


Epoch: 18, Train loss: 0.33603007942438123, Val metrics: {'mae': 0.24637329929831664}
Early stop at epoch 18


8

In [14]:
# test
net.load_state_dict(state_dict)
logits = test(task_loader_dict["test"], net, task_a)
logits = logits.numpy()
pred_hat = task_a.get_table("test", mask_input_cols=False).df[task_a.target_col].to_numpy()
test_metrics = {
        "mae": mean_absolute_error(pred_hat, logits),
        "r2": r2_score(pred_hat, logits),
        "rmse": root_mean_squared_error(pred_hat, logits),
}
test_metrics

{'mae': 0.2528144338551578,
 'r2': 0.04830285104458143,
 'rmse': 0.6266905023782507}